In [1]:
import time
import pandas as pd
import spacy
from rouge_score import rouge_scorer
import sacrebleu
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from transformers import MarianMTModel, MarianTokenizer

# Chargement spacy pour tokenization
nlp = spacy.load("en_core_web_sm")

def tokenize(text):
    return [token.text for token in nlp(text)]

def evaluate(reference, generated):
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    bleu = sacrebleu.sentence_bleu(" ".join(gen_tokens), [" ".join(ref_tokens)]).score

    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, generated)

    return bleu, scores["rouge1"].fmeasure, scores["rougeL"].fmeasure

# Chargement modèle Hugging Face FR->EN
hf_model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer_hf = MarianTokenizer.from_pretrained(hf_model_name)
model_hf = MarianMTModel.from_pretrained(hf_model_name)

def generate_hf(texts, temperature=0.0):
    inputs = tokenizer_hf(texts, return_tensors="pt", padding=True)
    outputs = model_hf.generate(
        **inputs,
        do_sample=True if temperature > 0 else False,
        temperature=temperature
    )
    translations = [tokenizer_hf.decode(t, skip_special_tokens=True) for t in outputs]
    return translations

def create_ollama_model(model_name, temperature):
    return ChatOllama(model=model_name, temperature=temperature)

def generate_ollama(model, text):
    prompt = f"Translate this French sentence to English:\n\n\"{text}\""
    try:
        response = model.invoke([HumanMessage(content=prompt)])
        return response.content.strip()
    except Exception as e:
        print(f"Error during model call: {e}")
        return ""

def generate_translation(model_name, temperature, text, framework="ollama"):
    """
    framework: "ollama" or "hf"
    """
    if framework == "hf":
        # HuggingFace attend une liste
        return generate_hf([text], temperature=temperature)[0]
    elif framework == "ollama":
        model = create_ollama_model(model_name, temperature)
        return generate_ollama(model, text)
    else:
        raise ValueError("framework must be 'ollama' or 'hf'")

# --- Paramètres ---

models_ollama = ["tinyllama:1.1b", "qwen2.5:0.5b"]
models_hf = ["Helsinki-NLP/opus-mt-fr-en", "facebook/wmt19-fr-en"]
temperatures = [0, 0.2, 0.5, 0.8, 1.0]

# Exemple de DataFrame avec colonnes : 'fr' (phrase française) et 'en' (référence anglaise)
df_data = pd.read_csv("sample_fr_en_clean.csv").head(50)

results = []

for i, row in df_data.iterrows():
    original = row["fr"]
    reference = row["en"]

    # Ollama
    for model_name in models_ollama:
        for temp in temperatures:
            print(f"Ollama - Model: {model_name} | Temp: {temp} | Example {i+1}/{len(df_data)}")
            generated = generate_translation(model_name, temp, original, framework="ollama")
            bleu, rouge1, rougeL = evaluate(reference, generated)
            results.append({
                "example_id": i,
                "model": model_name,
                "temperature": temp,
                "original": original,
                "reference": reference,
                "translation": generated,
                "BLEU": bleu,
                "ROUGE-1": rouge1,
                "ROUGE-L": rougeL,
                "framework": "ollama"
            })
            time.sleep(1)  # éviter surcharge

    # Hugging Face
    for model_name in models_hf:
        for temp in temperatures:
            print(f"Hugging Face - Model: {model_name} | Temp: {temp} | Example {i+1}/{len(df_data)}")
            generated = generate_translation(model_name, temp, original, framework="hf")
            bleu, rouge1, rougeL = evaluate(reference, generated)
            results.append({
                "example_id": i,
                "model": model_name,
                "temperature": temp,
                "original": original,
                "reference": reference,
                "translation": generated,
                "BLEU": bleu,
                "ROUGE-1": rouge1,
                "ROUGE-L": rougeL,
                "framework": "huggingface"
            })

df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_translations.csv", index=False)

print("✅ Évaluation terminée et sauvegardée dans evaluation_translations.csv")


c:\Users\Inaya\Documents\GitHub\projet_transalation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Inaya\Documents\GitHub\projet_transalation\venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Ollama - Model: tinyllama:1.1b | Temp: 0 | Example 1/50


C:\Users\Inaya\AppData\Local\Temp\ipykernel_12116\3679597211.py:43: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  return ChatOllama(model=model_name, temperature=temperature)


Ollama - Model: tinyllama:1.1b | Temp: 0.2 | Example 1/50
Ollama - Model: tinyllama:1.1b | Temp: 0.5 | Example 1/50
Ollama - Model: tinyllama:1.1b | Temp: 0.8 | Example 1/50
Ollama - Model: tinyllama:1.1b | Temp: 1.0 | Example 1/50
Ollama - Model: qwen2.5:0.5b | Temp: 0 | Example 1/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.2 | Example 1/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.5 | Example 1/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.8 | Example 1/50
Ollama - Model: qwen2.5:0.5b | Temp: 1.0 | Example 1/50


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0 | Example 49/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.2 | Example 49/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.5 | Example 49/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.8 | Example 49/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 1.0 | Example 49/50
Ollama - Model: tinyllama:1.1b | Temp: 0 | Example 50/50
Ollama - Model: tinyllama:1.1b | Temp: 0.2 | Example 50/50
Ollama - Model: tinyllama:1.1b | Temp: 0.5 | Example 50/50
Ollama - Model: tinyllama:1.1b | Temp: 0.8 | Example 50/50
Ollama - Model: tinyllama:1.1b | Temp: 1.0 | Example 50/50
Ollama - Model: qwen2.5:0.5b | Temp: 0 | Example 50/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.2 | Example 50/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.5 | Example 50/50
Ollama - Model: qwen2.5:0.5b | Temp: 0.8 | Example 50/50
Ollama - Model: qwen2.5:0.5b | Temp: 1.0 | Example 50/50


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Hugging Face - Model: Helsinki-NLP/opus-mt-fr-en | Temp: 0 | Example 50/50
Hugging Face - Model: Helsinki-NLP/opus-mt-fr-en | Temp: 0.2 | Example 50/50
Hugging Face - Model: Helsinki-NLP/opus-mt-fr-en | Temp: 0.5 | Example 50/50
Hugging Face - Model: Helsinki-NLP/opus-mt-fr-en | Temp: 0.8 | Example 50/50
Hugging Face - Model: Helsinki-NLP/opus-mt-fr-en | Temp: 1.0 | Example 50/50


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0 | Example 50/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.2 | Example 50/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.5 | Example 50/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 0.8 | Example 50/50
Hugging Face - Model: facebook/wmt19-fr-en | Temp: 1.0 | Example 50/50
✅ Évaluation terminée et sauvegardée dans evaluation_translations.csv
